In [1]:
# Load Libraries
import pandas as pd
import pickle
from pandas import DataFrame, Series
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from sklearn import linear_model, naive_bayes
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
import pybaseball as pyb
from pybaseball import statcast
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from keras.models import Sequential
from keras.layers import Dense
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

In [108]:
# Import Data
df = pd.read_csv("/Users/genoc/Desktop/HyannisHarborHawks/Trackman/CapeTrackmanDatabase.csv",low_memory=False)

# Clean Trackman Data
dft = df[['Pitcher', 'PitcherTeam', 'Batter', 'BatterTeam', 'PitcherThrows', 'RelSpeed','SpinRate','RelHeight','RelSide','Extension','InducedVertBreak', 'HorzBreak', 'PitchCall', 'PlayResult', 'TaggedHitType', 'KorBB', 'PitchofPA']]

#Rename the columns to be more descriptive
dft.rename(columns={'BatterTeam':'Batter Team', 'PitcherTeam':'Pitcher Team', 'PitcherThrows':'P Side', 'RelSpeed':'Velocity', 'SpinRate':'Spin Rate', 'RelHeight':'Release Height', 'RelSide':'Release Side', 'Extension':'Extension', 'InducedVertBreak':'Induced Vertical Break', 'HorzBreak':'Horizontal Break', 'PitchCall':'Pitch Call', 'PlayResult':'Play Result', 'TaggedHitType':'Hit Type'}, inplace=True)

#Add the Pitch Type and Pitch Result to the new dataframe
dft.insert(1, 'Pitch Type', df['TaggedPitchType'])
dft.insert(2, 'Pitch Result', df['PitchCall'])

#Create a new column for the difference between the Induced Vertical Break and the Horizontal Break (Absolute Value)
dft['Differential Break'] = (dft['Induced Vertical Break'] - dft['Horizontal Break']).abs()

#Add two new columns for Pitch Location
dft.insert(10, 'Pitch Height', df['PlateLocHeight'])
dft.insert(11, 'Pitch Side', df['PlateLocSide'])
dft.insert(12, 'Vertical Approach Angle', df['VertApprAngle'])
dft.insert(13, 'Horizontal Approach Angle', df['HorzApprAngle'])

#Edit Horizontal Break to be absolute value of horizontal break
dft['Horizontal Break'] = abs(dft['Horizontal Break'])

dft['Release Side'] = abs(dft['Release Side'])

# Edit the pitcher names so the first name is first and the last name is last
dft['Pitcher'] = dft['Pitcher'].str.split(' ', expand=True)[1] + ' ' + dft['Pitcher'].str.split(' ', expand=True)[0]

# Edit the batter names so the first name is first and the last name is last
dft['Batter'] = dft['Batter'].str.split(' ', expand=True)[1] + ' ' + dft['Batter'].str.split(' ', expand=True)[0]

# Remove the comma from the last name
dft = dft.replace(',', '', regex=True)

C:\Users\genoc\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-108-0568b319a265>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft['Differential Break'] = (dft['Induced Vertical Break'] - dft['Horizontal Break']).abs()
<ipython-input-108-0568b319a265>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

In [109]:
# Calculate the number of plate appearances, walks, strikeouts, popups, flyballs, and groundballs for each pitcher
pitcher_stats = dft.groupby('Pitcher').agg(
    Walks=('KorBB', lambda x: (x == 'Walk').sum()),
    Strikeouts=('KorBB', lambda x: (x == 'Strikeout').sum()),
    Popups=('Hit Type', lambda x: (x == 'Popup').sum()),
    Flyballs=('Hit Type', lambda x: (x == 'FlyBall').sum()),
    Groundballs=('Hit Type', lambda x: (x == 'GroundBall').sum()),
    Linedrives=('Hit Type', lambda x: (x == 'LineDrive').sum())
).reset_index()

pitcher_pa = dft[dft['PitchofPA'] == 1].groupby('Pitcher').agg(
    PlateAppearances=('PitchofPA', 'count')
).reset_index()

# Rename columns for clarity
pitcher_stats.rename(columns={
    'Walks': 'Total Walks',
    'Strikeouts': 'Total Strikeouts',
    'Popups': 'Total Popups',
    'Flyballs': 'Total Flyballs',
    'Groundballs': 'Total Groundballs',
    'Linedrives': 'Total Linedrives'
}, inplace=True)

# Create the df_siera dataframe
df_siera = pd.DataFrame(pitcher_stats)

In [110]:
df_siera['Plate Appearances'] = df_siera['Total Walks'] + df_siera['Total Strikeouts'] + df_siera['Total Popups'] + df_siera['Total Flyballs'] + df_siera['Total Groundballs'] + df_siera['Total Linedrives'] 

comp1 = 16.986 * (df_siera['Total Strikeouts'] / df_siera['Plate Appearances'])
comp2 = 11.434 * (df_siera['Total Walks'] / df_siera['Plate Appearances'])
comp3 = 1.858 * ((df_siera['Total Groundballs'] - df_siera['Total Flyballs'] - df_siera['Total Popups']) / df_siera['Plate Appearances'])
comp4 = 7.653 * pow((df_siera['Total Strikeouts'] / df_siera['Plate Appearances']), 2)
comp5 = 6.664 * pow(((df_siera['Total Groundballs'] - df_siera['Total Flyballs'] - df_siera['Total Popups']) / df_siera['Plate Appearances']), 2)
comp6 = 10.130 * (df_siera['Total Strikeouts'] / df_siera['Plate Appearances']) * ((df_siera['Total Groundballs'] - df_siera['Total Flyballs'] - df_siera['Total Popups']) / df_siera['Plate Appearances'])
comp7 = 5.195 * (df_siera['Total Walks'] / df_siera['Plate Appearances']) * ((df_siera['Total Groundballs'] - df_siera['Total Flyballs'] - df_siera['Total Popups']) / df_siera['Plate Appearances'])

df_siera['SIERA'] = 6.145 - comp1 + comp2 - comp3 + comp4 + comp5 + comp6 - comp7

In [111]:
df_siera['SIERA'].describe()

count    361.000000
mean       4.667398
std        2.274350
min       -2.669000
25%        3.310229
50%        4.464751
75%        5.557719
max       16.281889
Name: SIERA, dtype: float64

In [114]:
siera_df_sorted = df_siera.sort_values(by='SIERA', ascending=True)
siera_df_sorted = siera_df_sorted[(siera_df_sorted['Plate Appearances'] >= 6)]

In [115]:
siera_df_sorted['SIERA'].mean()

4.497177664056897

In [51]:
# SIERA MEAN VALUES BY TEAM

# Hyannis: 3.74
# Cotuit: 4.27
# Falmouth: 3.99
# Bourne: 3.76
# Wareham: 5.38
# Brewster: 4.62
# YD: 4.30
# Orleans: 4.95
# Chatham: 4.65
# Harwich: 3.85